# Seaborn time plotting demo

How to plot datetime values in a seaborn graph.  We use a fictional event stream with begin and end events for the time slices.

In [ ]:
import datetime as dt
import numpy as np
import pandas as pd
import seaborn as sns
sns.set(style="white")

## Create a dataframe with time strings and labels

https://stackoverflow.com/a/51308247/8928529

In [ ]:
data = np.array([["2020-05-06T00:10:33.000Z", 4, "1"], ["2020-05-06T00:10:59.000Z", 7, "2"], ["2020-05-06T00:10:59.000Z", 3, "1"], ["2020-05-06T00:11:43.000Z",7, "2"],["2020-05-06T00:11:43.000Z",5, "1"]])
data

In [ ]:
df = pd.DataFrame({'time': data[:, 0], 'labels': data[:, 1], 'event': data[:, 2]})
print(df)

## Plot Relative Time

You can  plot time events as a string but it doesn't have relative distance.
It's just plotting one event after the next.  
That's ok for showing sequence of events in the sorted data set but doesn't convey the span of association with an AP. 

It will be better to have the x axis account for the time distance between events.  The relplot() doesn't work with real datetime values.  The [recommendation is to convert to unix time in seconds](https://stackoverflow.com/a/44572082/8928529) for plotting and then display the x-axis labels with the associated dates.


We need to [convert the string representation of time to a datetime type for proper time computation](https://stackoverflow.com/a/32204679/8928529). Let's add the column and sort on it.

In [ ]:
df['datetime'] = pd.to_datetime(df.time)

In [ ]:
df.head(5)

Convert to unix time by [subtracting the epoch date](https://stackoverflow.com/a/54313505/8928529).  Use floor division (//) for whole seconds.

In [ ]:
df['utime'] = (df['datetime'] - pd.Timestamp("1970-01-01T00:00:00.000Z")) // pd.Timedelta('1s') 

The unix time will get correct relative time positions but to make the labels easier to read we need to convert the values from seconds to real time.  Seaborn plots return a handle that can be used to access the matplotlib.pyplot.axes object.  This can be used to [customize the values of the tickmarks on the x-axis back to dates](https://stackoverflow.com/a/52234951/8928529).

In the plot below we look at a very short sequence of events to make it easier to see the reationships between the association success and deauth from station messages.  We use right arrows for association success (begin) events and left arrows for deauth (end) events.

It's worth noting that [you can pass additional matplotlib parameters in seaborn plots and they get passed to the underlying plot functions](https://stackoverflow.com/a/52785672/8928529). Here we make the marker size bigger so the arrows are easier to read (s=100).

In [ ]:
def timeticks(ax, tformat="%H:%M:%S\n%Y-%m-%d"):
    xticks = ax.get_xticks()
    xticks_dates = [dt.datetime.fromtimestamp(x).strftime(tformat) for x in xticks]
    hush = ax.set_xticklabels(xticks_dates)

In [ ]:
# build the replot and capture the handle
g = sns.relplot(x="utime", y="labels",
            hue="event", 
            style_order=['1',"2"],
            style="event",
            markers="><",
            palette="bright",
            aspect=2,
            data=df[:10],
            s=100)

# update the axis labels
g = (g.set_axis_labels("Date", "AP"))

# update the x tickmarks from unix time to hour minute seconds
ax = g.axes
ax = ax[0,0]
timeticks(ax)